# MACD STRATEGY

Moves across the zero line on the indicator represent times when the 12day EMA is crossing the 26day EMA. When the MACD crosses the zero line from below, a new uptrend may be emerging. When the MACD crosses the zero line from above a new downtrend may be emerging.

In [1]:
%matplotlib inline  

import pandas as pd
import talib as tl
import numpy as np

In [2]:
def load(file):
    df = pd.read_csv(file, skiprows=1)
    return df.reindex(index=df.index[::-1]).reset_index()

In [3]:
datasets = {
    "BTCUSD": load('data/Bitstamp_BTCUSD_1h.csv'),
    "ETHUSD": load('data/Bitstamp_ETHUSD_1h.csv'),
    "LTCUSD": load('data/Bitstamp_LTCUSD_1h.csv'),
    "XRPUSD": load('data/Bitstamp_XRPUSD_1h.csv'),
    "ETHBTC": load('data/Binance_ETHBTC_1h.csv'),
    "LTCBTC": load('data/Binance_LTCBTC_1h.csv'),
    "XMRBTC": load('data/Binance_XMRBTC_1h.csv')
}

In [4]:
def benchMACD(df):
    df = df.copy()
    df['macd'] = tl.EMA(df.Close, 12) - tl.EMA(df.Close, 26)
    df['prevmacd'] = df.macd.shift()
    df = df.dropna()
    dd = df[((df.macd >0) & (df.prevmacd < 0)) | (df.macd <0) & (df.prevmacd > 0)]

    rows = []
    positionTake = False
    for i,e in dd.iterrows():
        if e.macd > 0 and e.prevmacd < 0:
            # open position
            positionTake = True
            position = e  
        if e.macd < 0 and e.prevmacd > 0:
            if positionTake:
                # close position
                rows.append({
                    "enter": position.Date,
                    "exit": e.Date,
                    "buyprice": position.Close,
                    "sellprice": e.Close,
                    "profit": (e.Close - position.Close) / position.Close,
                })
    return pd.DataFrame(rows)

In [5]:
results = {}
for e in datasets:
    roi = benchMACD(datasets[e])
    results[e] = { "roi": roi, "roiAvg": roi.profit.mean() }
    

In [6]:
for e in results:
    print(e, 'ntrades', len(results[e]['roi']), ', mean roi',results[e]['roiAvg'])

('XMRBTC', 'ntrades', 249, ', mean roi', -0.005390109992458716)
('BTCUSD', 'ntrades', 238, ', mean roi', 0.008968253655314026)
('LTCUSD', 'ntrades', 259, ', mean roi', 0.009241163237665374)
('ETHBTC', 'ntrades', 256, ', mean roi', 0.004687762014716199)
('LTCBTC', 'ntrades', 263, ', mean roi', 0.0038596112095064428)
('XRPUSD', 'ntrades', 264, ', mean roi', 0.012458220599033426)
('ETHUSD', 'ntrades', 213, ', mean roi', 0.007188110005702256)


In [7]:
results['XRPUSD']['roi']

,buyprice,enter,exit,profit,sellprice
0,0.2515,2017-07-06 12-AM,2017-07-06 02-AM,-0.007157,0.2497
1,0.2509,2017-07-06 09-PM,2017-07-07 03-AM,-0.013950,0.2474
2,0.2370,2017-07-09 12-AM,2017-07-09 03-PM,-0.029114,0.2301
3,0.1884,2017-07-12 11-AM,2017-07-13 05-PM,-0.026008,0.1835
4,0.1596,2017-07-17 08-AM,2017-07-17 11-AM,-0.063910,0.1494
5,0.1624,2017-07-17 05-PM,2017-07-19 05-PM,0.057882,0.1718
6,0.1810,2017-07-20 12-PM,2017-07-22 12-AM,0.011602,0.1831
7,0.1860,2017-07-22 01-AM,2017-07-24 06-AM,0.032258,0.1920
8,0.1954,2017-07-24 02-PM,2017-07-24 03-PM,-0.007677,0.1939
9,0.1760,2017-07-27 05-AM,2017-07-27 02-PM,-0.034091,0.1700
